In [ ]:
%pip install pint
%pip install pint_pandas
%pip install openpyxl
%pip install matplotlib
%pip install numpy

In [1]:
import matplotlib.pyplot as plt
import numpy as np

import pint
import pint_pandas

import pandas as pd

pd.options.mode.copy_on_write = True


class LabCtx:
    _ureg = pint.UnitRegistry()
    _Q = _ureg.Quantity
    pint_pandas.PintType.ureg = _ureg

    _ureg.enable_contexts("Gaussian")
    

    def __init__(self, exelPath: str) -> None:
        self._labData = pd.read_excel(exelPath, sheet_name=None, header=[0, 1])

    def _getTable(self, name: str) -> pd.DataFrame:
        return self._labData[name].pint.quantify(level=-1)

    def _toLatexFormat(self):
        self._ureg.default_format = "Lx"

    def _toDefaultFormat(self):
        self._ureg.default_format = "P"


Q = LabCtx._Q

UR = LabCtx._ureg


def q_for_eval(magnitude, unit=None):
    return Q(magnitude, unit)


class LabTable:
    def __init__(
        self, ctx: LabCtx, name: str = "", table: pd.DataFrame = pd.DataFrame(data={})
    ) -> None:

        self.ctx = ctx

        if not table.empty:
            self._table = table
            return

        self._table = self.ctx._getTable(name)

    def toLatex(
        self, file: str, columnsRen={}, caption="", float_format="%.2f", toStdout=False
    ) -> None:
        self.ctx._toLatexFormat()

        self._table.rename(columns=columnsRen).pint.dequantify().to_latex(
            file,
            index=False,
            caption=caption,
            position="H",
            float_format=float_format,
        )

        self.ctx._toDefaultFormat()

        if toStdout:
            print(self._table.rename(columns=columnsRen).pint.dequantify())

    def setColUnits(self, col: str, u: pint.Unit | str):
        self._table[col] = self._table[col].pint.to(u)
        return self

    def getColUnits(self, col: str) -> pint.Unit:
        return self._table[col].pint.units

    def addCol(self, col: str, data: pd.Series) -> None:
        self._table[col] = data

    def col(self, col: str) -> pd.Series:
        return self._table[col]

    def colQuantity(self, col: str) -> pint.Quantity:
        return self._table[col].values.quantity

    def colRaw(self, col: str) -> list[float]:
        return self._table[col].values.quantity.magnitude

    def query(self, q: str):
        return LabTable(self.ctx, table=self._table.query(q))

    def printPretty(self):
        print(self._table)


class LabPlot:
    def __init__(self, file: str, xlbl: str, ylbl: str) -> None:
        self.file = file

        self.xlbl = xlbl
        self.ylbl = ylbl

    def __enter__(self):
        self.fig, self.axis = plt.subplots(figsize=(12, 10), dpi=100)
        self.axis.grid(True)

        self.axis.set_xlabel(self.xlbl, fontsize=20)
        self.axis.set_ylabel(self.ylbl, fontsize=20)

        return self

    def __exit__(self, exc_type, exc_val, exc_tb):
        self.axis.legend()
        self.fig.savefig(self.file)


def plotMNK(ax: plt.Axes, x_units, y_units, name, scatter=False, prec=5) -> list:
    x = x_units
    y = y_units

    fit, pcov = np.polyfit(x, y, 1, full=False, cov=True)
    perr = np.sqrt(np.diag(pcov))
    poly = np.poly1d(fit)

    x = np.linspace(min(x), max(x), 50)
    ax.plot(
        x,
        poly(x),
        label=f"{name}: ${fit[0]:.{prec}f}\pm{perr[0]:.{prec}f}$",
    )

    if scatter:
        ax.scatter(x_units, y_units)

    return fit

In [ ]:
labCtx = LabCtx("lab4.3.1.xlsx")

In [ ]:
tbl1 = LabTable(labCtx, "1")

In [ ]:
tbl1.printPretty()

In [ ]:
tbl1.addCol("m", tbl1.col("n") + 1)

tbl1.addCol("2E", 2 * (tbl1.col("m") * tbl1.col("z") * Q("546 nm")) ** (1 / 2))

tbl1.setColUnits("2E", "mm")

In [ ]:
tbl1.toLatex(
    "tex/tables/1.tex",
    {"2E": r"$2\xi$"},
    float_format="%.2f",
)

In [ ]:
tbl2 = LabTable(labCtx, "2")

In [ ]:
tbl2.toLatex(
    "tex/tables/2.tex",
    float_format="%.2f",
)

In [ ]:
with LabPlot("tex/pictures/2.png", r"n", r"$x, mm$") as plot:

    plotMNK(
        plot.axis,
        tbl2.colRaw("n"),
        tbl2.colRaw("x"),
        "k:",
        scatter=True,
    )
    plt.close()

In [ ]:
k = Q("0.26 mm")
lamb = Q("546 nm")
f2 = Q("10.8 cm")

b = (lamb / k) * f2

b.to("mm")

In [ ]:
dx = Q("33.5  micrometer")

d = f2 * lamb / dx
d.to("mm")

(2 * d / 10).to("mm")

In [ ]:
tbl1.addCol("D", tbl1.col("high") - tbl1.col("low"))

In [ ]:
tbl1.toLatex(
    "tex/tables/green.tex",
    {"low": r"$d_\text{низ}$", "high": r"$d_\text{верх}$"},
    float_format="%.3f",
)

In [ ]:
with LabPlot("tex/pictures/green.png", r"n", r"$D^2$") as plot:

    plotMNK(
        plot.axis,
        tbl1.colRaw("n"),
        np.power(tbl1.colRaw("D"), 2),
        ":",
        scatter=True,
    )
    plt.close()

In [ ]:
tbl2 = LabTable(labCtx, "rty")
tbl2.addCol("D1", tbl2.col("high1") - tbl2.col("low1"))
tbl2.addCol("D2", tbl2.col("high2") - tbl2.col("low2"))

In [ ]:
tbl2.toLatex(
    "tex/tables/rtyellow.tex",
    {
        "low1": r"$d_\text{низ}$",
        "high1": r"$d1_\text{верх}$",
        "low2": r"$d2_\text{низ}$",
        "high2": r"$d2_\text{верх}$",
    },
    float_format="%.3f",
)

In [ ]:
with LabPlot("tex/pictures/rtyellow.png", r"n", r"$D_1^2 + D_2^2$") as plot:

    plotMNK(
        plot.axis,
        tbl2.colRaw("n"),
        np.power(tbl2.colRaw("D1"), 2) + np.power(tbl2.colRaw("D2"), 2),
        ":",
        scatter=True,
    )
    plt.close()

In [ ]:
L = Q(0.12, "mm")

f = Q(110, "mm")

k = Q(460.0, "mm^2")


(L * k / (8 * f**2)).to("nm")

In [ ]:
with LabPlot("tex/pictures/rtyellow2.png", r"n", r"$D_1^2 - D_2^2$") as plot:

    plotMNK(
        plot.axis,
        tbl2.colRaw("n")[1:],
        (np.power(tbl2.colRaw("D1"), 2) - np.power(tbl2.colRaw("D2"), 2))[1:],
        ":",
        scatter=True,
    )
    plt.close()

In [ ]:
k = Q(2.0, "mm^2")

(L * k / (4 * f**2)).to("nm")

In [ ]:
tbl3 = LabTable(labCtx, "ny")
tbl3.addCol("D1", tbl3.col("high1") - tbl3.col("low1"))
tbl3.addCol("D2", tbl3.col("high2") - tbl3.col("low2"))

In [ ]:
tbl3.toLatex(
    "tex/tables/nyellow.tex",
    {
        "low1": r"$d_\text{низ}$",
        "high1": r"$d1_\text{верх}$",
        "low2": r"$d2_\text{низ}$",
        "high2": r"$d2_\text{верх}$",
    },
    float_format="%.3f",
)

In [ ]:
with LabPlot("tex/pictures/nyellow.png", r"n", r"$D_1^2 + D_2^2$") as plot:

    plotMNK(
        plot.axis,
        tbl3.colRaw("n"),
        np.power(tbl3.colRaw("D1"), 2) + np.power(tbl3.colRaw("D2"), 2),
        ":",
        scatter=True,
    )
    plt.close()

In [ ]:
L = Q(0.12, "mm")

f = Q(110, "mm")

k = Q(443.0, "mm^2")


(L * k / (8 * f**2)).to("nm")

In [ ]:
with LabPlot("tex/pictures/nyellow2.png", r"n", r"$D_1^2 - D_2^2$") as plot:

    plotMNK(
        plot.axis,
        tbl3.colRaw("n"),
        (np.power(tbl3.colRaw("D1"), 2) - np.power(tbl3.colRaw("D2"), 2)),
        ":",
        scatter=True,
    )
    plt.close()

In [ ]:
k = Q(1.35, "mm^2")

(L * k / (4 * f**2)).to("nm")

In [ ]:
r = (177.215 - 170.425) / 2

deg1 = r / 110